In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:

import Libs.preprocess as preprocess
from Libs.Model import Net
from Libs.train import train_model
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import numpy as np
np.random.seed(0)
torch.manual_seed(0)

In [37]:
dir_training = "./data/training_data/"
dir_testing = "./data/testing_data/"

In [38]:
# data = preprocess.Data()
# df = data.load_label("data/training_labels.csv")
# df.sort_values(by=df.columns[0], inplace=True) # sort by id
# print(df.head(3))
# print(data.get_label(1))

## Dataset preprocess

In [39]:
# data.train = data.load_dir(dir_training)
# data.test = data.load_dir(dir_testing)

train_trans = transforms.Compose([transforms.Resize((400, 400)),
                                 transforms.ToTensor(), # range [0, 255] -> [0.0,1.0]
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.ImageFolder(root=dir_training, transform=train_trans)
train_loader =  torch.utils.data.DataLoader(train_dataset, batch_size = 32, shuffle=True)



## Model

In [40]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold = 0.9)
model_ft, training_losses, training_accs, test_accs = train_model(model, train_loader, criterion, optimizer, lrscheduler, 1)


KeyboardInterrupt: 